## Create FHIR R4 SearchParameter Resource

Create FHIR R4 SearchParameter Resource, Quick start text, and Searchparameter list using the python fhir client

Source data is in excel file

### Prerequisites:

- Python 3.7 or greater

### Import FHIRR4Client and other libraries

In [ ]:
%config IPCompleter.greedy=True

In [34]:
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from fhirclient.r4models import searchparameter as SP
from fhirclient.r4models import capabilitystatement as CS
from fhirclient.r4models import bundle as B
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from IPython.display import display as Display, HTML, Markdown
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape
import R4sp_summary_list as sp_map
from stringcase import snakecase, titlecase, pascalcase
from itertools import zip_longest
from openpyxl import load_workbook

####  Assign Global Variables


Here is where we assign all the global variables for this example such as the local paths for file input and output

##### Need to update:
- base_id
- paths
- canonical

In [ ]:
#******************** Need to update *************************************************
fhir_base_url = 'http://hl7.org/fhir/'
base_id = "US-Core-R4"
canon_base = "http://hl7.org/fhir/us/core/"
ig_folder = 'US-Core'
publisher = 'HL7 International - Structured Documents Work Group'
publisher_endpoint = dict(
                        system = 'url',
                        value = 'http://www.hl7.org/Special/committees/structure/index.cfm'
                        ) 

#***********************************************************************************

md_template = ['search_narrative.j2', 'sp_list_page.j2', 'cs_search_documentation.j2']


fhir_term_server = 'http://test.fhir.org/r3'

# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient



none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False', 'FALSE']
sep_list = (',', ';', ' ', ', ', '; ')



### validate

In [ ]:
# *********************** validate Resource as Dict ********************************

def validate(r):

    fhir_test_server = 'http://test.fhir.org/r4'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    
    params = {
    }
    
    r = post(f'{fhir_test_server}/{r["resourceType"]}/$validate', params = params, headers = headers, data = dumps(r))
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Write to File

In [ ]:
 def write_file(name, data): # write file
    
    out_path = ''
    #out_path = f'//ERICS-AIR-2/ehaas/Documents/FHIR/{ig_folder}/source/resources/'
    with open(f'{Path(out_path)}/{name}.json', 'w') as f:
        f.write(data)

### Get Search Parameter input data

In [ ]:
#in_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
in_path =''

in_file ="uscore-server"

xls = ExcelFile(f'{in_path}{in_file}.xlsx')
df = read_excel(xls,'sps',na_filter = False)

df

In [ ]:
data = [i for i in df.itertuples(index=True)]
r_type =  {d.base for d in data}
for d in data:
    print(f'Rresource = {d.base}, Search Parameter = {d.code}, Exists = {d.exists}')

### update core SP with additional capabiliities


- Get definitions bundle and convert to python object for ease of notation
- use sp_map to map to Type + parameter
- If need to update SP Extract the SP based on the excel file

### load SP Mapping dictionary

In [ ]:
p = Path('C:/Users/Eric/Documents/HL7/FHIR/BUILD_EDIT_FILES/R4_Definitions/search-parameters.json')

b = B.Bundle(loads(p.read_text()), strict = False)
sp = b.entry[0]
print(sp.fullUrl)
pprint([i for i in sp_map.sp_list2 if i[0]=="Observation" and i[1] == 'code'])
sp.as_json()

## create updated SPs
- in spreadsheet sp.elements as comma separated list and sp/element_conf as comma separated list
- if not listed then conformance is MAY 

In [36]:
def sp_expectation(conf=None):
    if not conf:
        conf = "MAY"
 
    x = X.Extension(dict(
    url = f'http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation',
    valueCode = conf
    ))
    x_dict = dict(
    extension = [x.as_json()]
    )
 
    
    return x_dict


sp_list=[]
for d in data:
    for i in b.entry:
        sp = i.resource
        if d.update =='Y' and d.base in sp.base and d.code == sp.code: #update sp
            # print(dumps(i.resource.as_json(),indent = 3))
            # change id and url, publisher, and contact, draft etc
            sp.derivedFrom =sp.url
            sp.url = f'{canon_base}SearchParameter/{sp.id}'
            sp.publisher = publisher
            sp.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
            sp.date = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
            sp.name = pascalcase(sp.id.replace('-','_'))
            sp.status = 'active'
            #convert to dict since model can't handle primitive extensions
            sp_dict = sp.as_json()

            sp_dict['multipleOr'] = False if d.multipleOr in none_list else True
            sp_dict['_multipleOr'] = sp_expectation(d.multipleOr_conf)
            
            sp_dict['multipleAnd'] = False if d.multipleAnd in none_list else True
            sp_dict['_multipleAnd'] = sp_expectation(d.multipleAnd_conf)

            try:
                sp_dict['_modifier'] = []
                for m in sp_dict['modifier']: # list all modifiers in sp and assign an expectation.
                    if d.shall_modifier not in none_list:
                       sp_dict['_modifier'].append(sp_expectation('SHALL'))
                    elif  d.should_modifier not in none_list:
                        sp_dict['_modifier'].append(sp_expectation('SHOULD'))               
                    else:
                        sp_dict['_modifier'].append(sp_expectation('MAY'))
            except KeyError:
                del(sp_dict['_modifier'])

            try:
                sp_dict['_comparator'] = []
                for m in sp_dict['comparator']: # list all comparators in sp and assign an expectation.
                   if d.shall_comparator not in none_list:
                       sp_dict['_comparator'].append(sp_expectation('SHALL'))
                   elif  d.should_comparator not in none_list:
                        sp_dict['_comparator'].append(sp_expectation('SHOULD'))               
                   else:
                        sp_dict['_comparator'].append(sp_expectation('MAY'))
            except KeyError:
                del(sp_dict['_comparator'])

            if d.shall_chain not in none_list:
               sp_dict['chain'] = d.shall_chain.split(',')
               sp_dict['_chain'] = [sp_expectation('SHALL') for c in d.shall_chain.split(',')]

            if d.should_chain not in none_list:
               sp_dict['chain'] = d.should_chain.split(',')
               sp_dict['_chain'] = [sp_expectation('SHALL') for c in d.should_chain.split(',')]

            print(f'======================= SP = {sp_dict["id"]} =====================')
            print(dumps(sp_dict,indent=4))
            sp_list.append(sp_dict)

======================= SP = Questionnaire-status =====================
{
    "id": "Questionnaire-status",
    "extension": [
        {
            "url": "http://hl7.org/fhir/StructureDefinition/structuredefinition-standards-status",
            "valueCode": "trial-use"
        }
    ],
    "base": [
        "Questionnaire"
    ],
    "code": "status",
    "contact": [
        {
            "telecom": [
                {
                    "system": "url",
                    "value": "http://www.hl7.org/Special/committees/structure/index.cfm"
                }
            ]
        }
    ],
    "date": "2019-04-23T17:56:54.351330Z",
    "derivedFrom": "http://hl7.org/fhir/us/core/SearchParameter/Questionnaire-status",
    "description": "The current status of the questionnaire",
    "experimental": false,
    "expression": "Questionnaire.status",
    "modifier": [
        "missing",
        "text",
        "not",
        "in",
        "not-in",
        "below",
        "above",
   

### Validate

In [37]:
for i in sp_list:
    
    r = validate(i)
    display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))               

### Create Ad Hoc Structure for listing all the required Search Parameters

- uses Spreadsheet
- uses set theory and math to enumarate combinations
- uses exclude list to remove prohibited combination
- uses least strong Conformance of members of combot to determine conformance


create a Dict with key = Resource_type and value equal to a sorted list of tuples with combonent parameter list, conformance  e.g. {Type:[(p_combo-1,c1),(p_combo2,c2...]}

add new sheet to spreadsheet for manual review

In [38]:
def get_conf_str(combo, r_type):
    conf_str = ''
    for k in df.itertuples(index=True):
        if k.base == r_type and k.code in combo:
            if  k.base_conf == 'MAY' or k.base_conf in none_list:
                conf_str = 'MAY'
                break
            elif k.base_conf == 'SHOULD':
                conf_str = 'SHOULD' 
            elif k.base_conf == 'SHALL' and conf_str not in ['SHALL','MAY']:
                conf_str ='SHALL' 
    return conf_str

def get_type_list(combo, r_type):
    t = {i.type for i in df.itertuples(index=True) if i.code in combo and i.base == r_type}
    print(t)
    return list(t)


def get_combos(pairs,c_list):
    a_list =[]
    for i in c_list:
        for j in c_list:
            #print(f'i={i} j = {j}, i&j= {i&j} i^j= {i^j}')
            if i & j and i != j:
                #print(f'i={i}, j = {j}, i&j= {i&j},i^j= {i^j} i|j = {i|j}')
                if i^j in pairs:
                    
                    if i|j not in a_list + c_list:
                        a_list.append(i|j)
    return a_list

          

combo_list = []
combo_dict = {}
for base in r_type:
    print(base)
    pairs = []
    sp_len = len([d for d in data if d.base== base])
          
    for d in data:
        if d.base == base and d.combo_pairs not in none_list: 
            pairs.extend([{d.code, p} for p in d.combo_pairs.split(',') if p != d.code])  # get allowed pairs
            #print(f'pairs = {pairs}')
          
    combo_list = pairs
    for j in range(sp_len-1):
         combo_list = combo_list + get_combos(pairs,combo_list)
          
    # convert to sorted tuples
    combo_list = [sorted(tuple(i)) for i in combo_list]
    combo_list = sorted(combo_list)
          
    try: #subtract forbidden combos
        f_combos = []
        df_r = read_excel(xls,'resources',na_filter = False)
        for r in df_r.itertuples(index=True):
            if  r.type == base:
                f_combos = r.forbidden_s_combos.split(',')
                f_combos = [i.split('|') for i in f_combos]
        # print(f_combos)
        combo_list = [i for i in combo_list if i not in f_combos]
    except AttributeError:
        print('forbidden_s_combos is missing')
          
    print(f'{len(combo_list)} combos = {combo_list}')
    combo_list.extend(combo_list)
    combo_dict[base]=[(c, get_conf_str(combo = c, r_type = base), get_type_list(combo = c, r_type = base)) for c in combo_list]
print(combo_dict)

Questionnaire
9 combos = [['context-type-value', 'publisher'], ['context-type-value', 'publisher', 'status'], ['context-type-value', 'status'], ['publisher', 'status'], ['publisher', 'status', 'version'], ['publisher', 'version'], ['status', 'title', 'version'], ['status', 'version'], ['title', 'version']]
{'composite', 'string'}
{'composite', 'string', 'token'}
{'composite', 'token'}
{'string', 'token'}
{'string', 'token'}
{'string', 'token'}
{'string', 'token'}
{'token'}
{'string', 'token'}
{'composite', 'string'}
{'composite', 'string', 'token'}
{'composite', 'token'}
{'string', 'token'}
{'string', 'token'}
{'string', 'token'}
{'string', 'token'}
{'token'}
{'string', 'token'}
QuestionnaireResponse
11 combos = [['author', 'patient'], ['author', 'patient', 'status'], ['author', 'status'], ['context', 'patient'], ['context', 'patient', 'status'], ['context', 'status'], ['patient', 'source'], ['patient', 'source', 'status'], ['patient', 'status'], ['questionnaire', 'status'], ['source',

### Convert combo_dict to dataframe and add to spreadsheet for review

In [42]:
rows = []
for k,v in combo_dict.items():
    for i1,i2,i3 in v:
        rows.append([k, '+'.join(i1),i2,i3])
        
df_combos= DataFrame(rows, columns=['Base Type',"Combo",'Conformance','Parameter Types'])

book = load_workbook(f'{in_path}{in_file}.xlsx')
with ExcelWriter(f'{in_path}{in_file}.xlsx', engine = 'openpyxl') as writer:
    writer.book = book
    df_combos.to_excel(writer, sheet_name = 'sp_combos')

xls = ExcelFile(f'{in_path}{in_file}.xlsx')
df = read_excel(xls,'sp_combos',na_filter = False)

#df

### Create Markdown Text for Search.

- Using Jinja2 Template create markdown file for search section in profiles profiles
- use spreadhsheet as input data  ( could use the sp too )

In [ ]:
#r_type = ['Questionnaire', 'QuestionnaireResponse', 'Patient']  # TODO  loop over all types in sp_list


search_type = dict(
    token = '{[system]}|[code]',
    id = '[id]',
    reference = '[url]',
    string = '[string]',
    uri = '[uri]',
    date = '[date]',
    )

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

template = env.get_template(md_template[0])

# use sp_dict to create the narratvie using Jinja2
'''
1. **{{i.base_conf}}** support search by the **`{{i.code}}`** parameter:

  `GET [base]/{{i.base[0]}}?{{i.code}}={{search_type[i.type]}}`
{%- endfor %}
for t in r_type:
    sp = [d2{%- endfor %}o(i) for i in sp_list if i['base'][0] in  r_type]
'''
    
for t in r_type:
    print(t)
    search_md = template.render(r_type=t,sp=data,search_type=search_type,combos=combo_dict[t])

    display(Markdown(search_md))


### Create Markdown Text for SearchParameters Page

- Using Jinja2 Template create markdown file for searchparameters page

In [ ]:
# get types:
r_list = []
for i in sp_list:
    for j in i['base']:
        r_list.append(j)

r_list = sorted(set(r_list))

# md_template = 'sp_list_page.j2'

template = env.get_template(md_template[1])

searchparameters_md = template.render(sp_list=sp_list,r_list=r_list)

display(Markdown(searchparameters_md))
